# 3D X-ray Histology VISualisation (3DXRH-Vis)

3DXRH-Vis is a tool for interactive visualisation of 2D images registered to a 3D dataset. It was designed for classical histology slides registered to 3D X-ray micro-computed tomography datasets, but should also work for any 2D-3D registered images.


In [1]:
# Set up, only needs to be run once at beginning
!conda activate CorView

import os, sys, time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import interact
from ipyfilechooser import FileChooser
from skimage import io, util
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, output_file
from bokeh.layouts import gridplot

## Import 3D image


In [3]:
class Image3D():
    """ Imports a 3D image when button is pressed """
    
    def __init__(self):
        pass
    
    def _create_widgets(self):
        self.select_file = FileChooser(os.getcwd())
        self.load_button = widgets.Button(description="Load Image")
        self.load_button.on_click(self._on_load_button_clicked)
    
    def _on_load_button_clicked(self, change):
        self.out.clear_output()
        with self.out:
            print("Importing image {}, please wait...".format(self.select_file.selected_filename))
            img_import = io.imread(self.select_file.selected, plugin="pil")
            
            # Is the image a 3D stack?
            if len(img_import.shape) != 3:
                raise ValueError("Image is not a 3D stack, please choose another image")
                
            # Convert to 8-bit RGB
            self.img = util.img_as_ubyte(img_import)
            print("Image successfully imported")
    
    def display_widgets(self):
        self._create_widgets()
        self.out = widgets.Output()
        display(self.select_file, self.load_button, self.out)
           
    def get_img(self):
        return self.img
    
    def get_img_filename(self):
        return self.select_file.selected_filename

def show_3D_slider(img):
    def show_slice(slice_number):
        io.imshow(img[slice_number])
        io.show()
    slider = widgets.IntSlider(min=0, max=len(img), step=1, description="Slice number")
    interact(show_slice, slice_number=slider)

Stack3D = Image3D()
Stack3D.display_widgets()


FileChooser(path='C:\Users\emlh1n13\OneDrive - University of Southampton\Projects\Aim2\CorView', filename='', …

Button(description='Load Image', style=ButtonStyle())

Output()

In [4]:
img = Stack3D.get_img()
fname_3d = Stack3D.get_img_filename()
print(fname_3d)
show_3D_slider(img)

HN1_test.tif


interactive(children=(IntSlider(value=0, description='Slice number', max=239), Output()), _dom_classes=('widge…

In [ ]:
reset

## Import registered 2D images into 3D stack.

Run the following cell to import a 2D image and specify the position in the 3D stack to import the 2D image.

The following cell inserts the 2D image at the specified position.

In [6]:
instances_2D = {}

class Image2D():
    """ Imports a 2D image and records instances """
    
    def __init__(self):
        self.time_imported = time.strftime("%D%H%M%S", time.localtime()) # saves the time as a unique ID
        instances_2D[self.time_imported] = self
    
    def _create_widgets(self):
        self.select_file = FileChooser(os.getcwd())
        self.load_button = widgets.Button(description="Load image")
        self.position = widgets.BoundedFloatText(min=0, max=len(img), step=1, description="Position of 2D image")
        self.load_button.on_click(self._on_load_button_clicked)
    
    def _on_load_button_clicked(self, change):
        self.out.clear_output()
        with self.out:
            print("Importing image {}, please wait...".format(self.select_file.selected))
            img_import = io.imread(self.select_file.selected, plugin='pil')
            if len(img_import.shape) != 3:
                raise ValueError("Image is not 2D, please choose a 2D image")
            self.img_2d = util.img_as_ubyte(img_import) # 8bit RGB
            print("Image successfully imported")
    
    def display_widgets(self):
        self._create_widgets()
        self.out = widgets.Output()
        display(self.select_file, self.position, self.load_button, self.out)
    
    def get_2D_img_fname(self):
        return self.select_file.selected_filename
    
    def get_2D_img(self):
        return self.img_2d
    
    def get_position(self):
        return int(self.position.value)

class CreateImage2D():
    """ Creates Image2D instances """
    def __init__(self):
        self.widget_layout = widgets.Layout(width='auto', height='40px')
    
    def _create_widgets(self):
        self.new_img_button = widgets.Button(description="Add new image", layout=self.widget_layout)
        self.new_img_button.on_click(self._import_2D)
        self.update_instances_button = widgets.Button(description="Display list of loaded images", 
                                                      layout=self.widget_layout)
        self.update_instances_button.on_click(self._display_instances)
        self.clear_all_button = widgets.Button(description="Clear all 2D images", layout=self.widget_layout)
        self.clear_all_button.on_click(self._clear_all)
        
    def _import_2D(self, change):
        """ Create instance of ImageImport2D, shows all instances """
        Img2D = Image2D()
        Img2D.display_widgets()
    
    def _display_instances(self, change):
        """ Displays all instances of ImageImport2D """
        clear_output()
        self.display_widgets()
        instance_2D_fnames = []
        instance_2D_positions = []
        for key in [*instances_2D.keys()]:
            instance_2D_fnames.append(instances_2D[key].get_2D_img_fname())
            instance_2D_positions.append(instances_2D[key].get_position())
        instance_2D_data = {'Key': [*instances_2D.keys()], 'Filenames': instance_2D_fnames, 
                            'Positions': instance_2D_positions}
        instance_2D_df = pd.DataFrame(data=instance_2D_data)
        display(instance_2D_df)
        
    def _clear_all(self, change):
        """ Deletes all instances of ImageImport2D """
        clear_output()
        self.display_widgets()
        with self.out:
            print("Clearing all 2D images")
            instances_2D.clear()
            print("Cleared all 2D images")
    
    def display_widgets(self):
        self._create_widgets()
        self.out = widgets.Output()
        hbox = widgets.HBox([self.new_img_button, self.update_instances_button, self.clear_all_button])
        vbox = widgets.VBox([hbox, self.out])
        display(vbox)

In [7]:
New2D = CreateImage2D()
New2D.display_widgets()

FileChooser(path='C:\Users\emlh1n13\OneDrive - University of Southampton\Projects\Aim2\CorView', filename='', …

BoundedFloatText(value=0.0, description='Position of 2D image', max=239.0, step=1.0)

Button(description='Load image', style=ButtonStyle())

Output()

# Side-by-side viewing

In [8]:
def rgba(img_2D, alpha=255):
    """ Convert RGB to RGBA image with alpha """
    shape = img_2D.shape
    alpha = np.full(shape=(shape[0],shape[1],1), fill_value=alpha, dtype='uint8') # create an alpha channel at full alpha
    img_rgba = np.concatenate((img_2D, alpha), axis=2) # join to rgb to make a rgba image

    return img_rgba

class CreateVis():
    """ Creates visualisations for given images """
    
    def __init__(self):
        if Stack3D is None:
            raise NameError("No CT image imported")
        else:
            self.img_3D = Stack3D
        self.widget_layout = widgets.Layout(width='auto', height='40px')
        self.widget_style = {'description_width': 'auto'}
    
    def _create_widgets(self):
        # Choose output filename
        self.output_fname_textbox = widgets.Text(value="3DXRH-Vis.html", description="Enter desired output filename",
                                                layout=self.widget_layout, style=self.widget_style)
        self.output_fname = self.output_fname_textbox.value
        if self.output_fname.endswith('.html') == False:
            self.output_fname+='.html' # ensures correct file extension
        self.output_fname_textbox.observe(self._on_change_output_filename, names='value')
        
        # Select one of 2D instances to display
        options = []
        for key in [*instances_2D.keys()]:
            options.append(("{} at position {}".format(instances_2D[key].get_2D_img_fname(), 
                                                            instances_2D[key].get_position()), key))
        self.instances_2D_dropdown = widgets.Dropdown(options=options, value=[*instances_2D.keys()][0],
                                                     description="2D image to display", layout=self.widget_layout,
                                                     style=self.widget_style)
        self.instances_2D_dropdown.observe(self._on_change_instances_2D, names='value')
        self.img_2D = instances_2D[self.instances_2D_dropdown.value]
    
        # Button to generate visualisation
        self.generate_vis_button = widgets.Button(description="Generate visualisation", layout=self.widget_layout,
                                                 style=self.widget_style)
        self.generate_vis_button.on_click(self._generate_plot)
          
    def _on_change_output_filename(self, change):
        """ Updates self.output_fname attribute when dropdown menu is changed """
        self.output_fname = change.new
        if self.output_fname.endswith('.html') == False:
            self.output_fname+='.html' # ensures correct file extension
    
    def _on_change_instances_2D(self, change):
        """ Updates self.img_2D attribute when dropdown menu is changed """
        self.img_2D = instances_2D[change.new]
    
    def _generate_plot(self, change):
        """ Bokeh plot of registered 2D images side-by-side with linked zooming and panning """
        img_CT = self.img_3D.get_img()
        img_histo = self.img_2D.get_2D_img()
        CT_slice = self.img_2D.get_position()
        tools = "pan, wheel_zoom, box_zoom, reset"
        
        # set up Bokeh
        output_file(self.output_fname, "Correlative micro-CT and histology")
        
        # show CT
        p_CT = figure(plot_width=500, plot_height=500, title="CT slice {}".format(str(CT_slice)), tools=tools)
        p_CT.image(image=[img_CT[int(CT_slice)]], x=[0], y=[0], dw=[img_CT.shape[1]], dh=[img_CT.shape[0]],palette="Greys256")

        # show histo
        p_histo = figure(plot_width=500, plot_height=500, x_range=p_CT.x_range, y_range=p_CT.y_range, 
                         title="Histology {}".format(self.img_2D.get_2D_img_fname()), tools=tools)
        p_histo.image_rgba(image=[rgba(img_histo)], x=[0], y=[0], dw=[img_CT.shape[1]], dh=[img_CT.shape[0]])

        p = gridplot([[p_CT, p_histo]])
        show(p)
        
    def display_widgets(self):
        self._create_widgets()
        self.out = widgets.Output()
        hbox = widgets.HBox([self.output_fname_textbox, self.instances_2D_dropdown])
        vbox = widgets.VBox([hbox, self.generate_vis_button, self.out])
        display(vbox)
        
bokeh_plot = CreateVis()
bokeh_plot.display_widgets()

# to do

- [done] Wrap 2D importing into a function so we can import several 2D images
- [done] Menu to choose 2D images to skip to
- [done] Side by side viewer
- [done] Linked pan and zoom for side-by-side
- [done] Max intensity
- Orthoslice viewer for combined stack
- [done] Export side-by-side plot to html
- Port to Voila

## dep


## Explore 3D dataset

3D images are displayed as sequences of 2D images. Run the following cell to import a 3D dataset.

In [ ]:
# Import a 3D dataset
fc_3d = FileChooser(os.getcwd())
display(fc_3d)

Run the following cell to view the 3D dataset.

In [ ]:
def show_3D_slider():
    img = Image.open(fc_3d.selected)
    
    def show_slice(slice_number):
        img.seek(slice_number)
        display(img)
        
    slider = widgets.IntSlider(min=0, max=img.n_frames, step=1, description="Slice number")
    interact(show_slice, slice_number=slider)
    
    return img

img = show_3D_slider()

In [ ]:
io.find_available_plugins()
